In [16]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-workspace
Azure region: eastus2
Subscription id: 11a5a317-f113-41bb-99e4-848e0ece5690
Resource group: udacity-resource-group


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)


compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

Creating a new compute cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-12-28T19:50:26.068000+00:00', 'errors': None, 'creationTime': '2023-12-28T19:50:18.001005+00:00', 'modifiedTime': '2023-12-28T19:50:27.799284+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}
Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisio

In [33]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

ps = RandomParameterSampling({
    '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
    '--max_iter': choice(100, 200, 300)  # Max iterations
})

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                      script='train.py',
                      arguments=['--C', 1, '--max_iter', 100],
                      compute_target=compute_target,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

In [34]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_aaa62801-ef82-40df-bbda-e952da36e3f3
Web View: https://ml.azure.com/runs/HD_aaa62801-ef82-40df-bbda-e952da36e3f3?wsid=/subscriptions/11a5a317-f113-41bb-99e4-848e0ece5690/resourcegroups/udacity-resource-group/workspaces/udacity-workspace&tid=e1ba80b0-c36a-48b5-b643-a9cc38ec5023

Streaming azureml-logs/hyperdrive.txt

[2023-12-28T20:47:24.876031][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-12-28T20:47:25.3628532Z][SCHEDULER][INFO]Scheduling job, id='HD_aaa62801-ef82-40df-bbda-e952da36e3f3_0' 
[2023-12-28T20:47:25.4921507Z][SCHEDULER][INFO]Scheduling job, id='HD_aaa62801-ef82-40df-bbda-e952da36e3f3_1' 
[2023-12-28T20:47:25.6201771Z][SCHEDULER][INFO]Scheduling job, id='HD_aaa62801-ef82-40df-bbda-e952da36e3f3_2' 
[2023-12-28T20:47:25.7247723Z][SCHEDULER][INFO]Scheduling job, id='HD_aaa62801-ef82-40df-bbda-e952da36e3f3_3' 
[2023-12-28T20:47:25.640829][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target

In [35]:
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))


[{'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_9', 'hyperparameters': '{"--C": 100, "--max_iter": 100}', 'best_primary_metric': 0.9088012139605463, 'status': 'Completed'}, {'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_10', 'hyperparameters': '{"--C": 20, "--max_iter": 100}', 'best_primary_metric': 0.9088012139605463, 'status': 'Completed'}, {'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_14', 'hyperparameters': '{"--C": 1000, "--max_iter": 300}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}, {'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_13', 'hyperparameters': '{"--C": 50, "--max_iter": 100}', 'best_primary_metric': 0.9084977238239758, 'status': 'Completed'}, {'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_7', 'hyperparameters': '{"--C": 1000, "--max_iter": 200}', 'best_primary_metric': 0.9084977238239758, 'status': 'Completed'}, {'run_id': 'HD_aaa62801-ef82-40df-bbda-e952da36e3f3_15', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'be

In [38]:
import joblib
from azureml.core import Model
# Get your best run and save the model from that run.

# Assuming hyperdrive_run is your HyperDriveRun object
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Parameters:', parameter_values)
print("Best run file names :",best_run.get_file_names())


Best Run Id:  HD_aaa62801-ef82-40df-bbda-e952da36e3f3_10

 Accuracy: 0.9088012139605463

 Parameters: ['--C', '1', '--max_iter', '100', '--C', '20', '--max_iter', '100']
Best run file names : ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-12-28-20', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [41]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])


In [46]:
!ls 'training'

train.py


In [50]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


In [63]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= ds,
    label_column_name= 'y',
    n_cross_validations= 2)

In [64]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_087423bc-96d9-444a-b234-8fcc3c3374bc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_087423bc-96d9-444a-b234-8fcc3c3374bc',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-12-28T21:25:53.917285Z',
 'endTimeUtc': '2023-12-28T22:12:36.112828Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"11a5a317-f113-41bb-99e4-848e0ece5690","resource_group":"udacity-resource-group","workspace_name":"udacity-workspace","region":"eastus2","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"

In [66]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

print("Best run metrics :",best_run.get_metrics())

print("Best run details :",best_run.get_details())

Run(Experiment: udacity-project,
Id: AutoML_087423bc-96d9-444a-b234-8fcc3c3374bc_37,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/zalfaraj1/code/Users/zalfaraj')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('30', Pipeline(steps=[('...m_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0.7291666666666667, reg_lambda=2.3958333333333335, subsample=0.8, tree_method='auto'))]))], flatten_transform=False, weights=[0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]))])
Y_transformer(['LabelEncoder', LabelEncoder()])
Best run metrics : {'recall_score_macro': 0.774

In [67]:
fitted_model


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/zalfaraj1/code/Users/zalfaraj')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=arr...cpu'}), reg_alpha=0.7291666666666667, reg_lambda=2.3958333333333335, subsample=0.8, tree_method='auto'))]))], flatten_transform=False, weights=[0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [68]:
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_087423bc-96d9-444a-b234-8fcc3c3374bc_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
compute_target.delete()